# Ch7. The CRSP Sample and Market Factor

Cause the data is needed for portfolio analysis, we start from the Ch7.   
CRSP data is neccesary for financial studieds 1925 through the present.   
In the book, as in almost all studies of the U.S. stock market, we use the data from CRSP as our primary source for U.S. stock market data.

## Import required

In [1]:
import pandas as pd 
import numpy as np

import wrds

c:\Users\YeonChan Kang\anaconda3\envs\fdb\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
db = wrds.Connection() # Connect to WRDS

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


crsp의 월간 주식이름 파일 msenames
공유코드: shrcd 10 or 11인 경우, crsp 미국 기반 보통주 sample 또는 crsp sample 이라 함

shrcd의 경우, first digit 1은 일반 보통주, second digit은 security type을 나타냄
10 - 더 이상 존재하지 않는 증권
11 - 일반적인 증권
12 - 미국 이외 지역 설립회사

거래소와 sector로 decomposition 가능

stock i의 t(month) 시점 시가총액 = 마지막 거래일 종료 시점의 주식가격 x 발행주식수 (단위: 1000)
MktCap = abs(ALTPRC * SHROUT / 1000) 
위 방법으로 계산 시, MktCap을 million 단위로 측정 가능
CRSP는 주식거래가 없는 경우, prc를 가장 최근의 매수 및 매도 가격 평균의 음수로 보고 하기 떄문에 절대값 사용

대부분 CPI(Consumer Price Index)를 사용하여 MktCap을 조정
시장 포트폴로이오의 총 가치는 는 거의 NYSE에서 발생하며, 최근 NASDAQ 비율이 증가중, AMEX에서 발생한 주식 가치는 매우 낮음

SIC 코드를 사용해 산업으로 그룹화 가능
SIC 코드는 1 ~ 9999 사이 숫자
SIC 코드는 msenames 파일의 SICCD에서 확인가능

모든 시점에서 제조업이 CRSP sample의 value에서 가장 큰 부분을 차지

월간 주식 수익률은 ret에서 확인 가능
예외 경우는 상장 폐지의 경우
이 때, 해당 월의 

msedelist
위 리스트에는 상장 폐지 날자 DLSDT
전월, 마지막 거래 가격으로 주식 매수 후 상장 폐지시 까지 주식을 보유한 투자자의 실현  수익률 DLRET
상장폐지 사유 DLSTCD 등이 기재되어 있음

DLRET은 상장폐지 수익률로 사용되며,
DLSTCD 의 상장폐지 사유가 500, 520, 551~573, 574, 580, or 584인 경우, 상장 폐지월 동안 주식수익률은 -30%로 간주함 (Shumway, 1997)

DLSTCD가 이외 라면, 상장폐지 수익률은 -100%

risk-free rate는 French library에서 사용

t기의 Excess return은 $r_{t+1}$, return은 $R_{t+1}$로 표기
값은 모두 백분율로 표기
1기 뒤는 t+1, 1기 전은 t-1로 표기

CRSP data는 1926년부터 존재하나, 대부분 연구는 1963년 6월 부터 사용가능한 마지막 연도를 포함하는 샘플을 사용
본 책에서는 1963년부터, 2012년 12월까지 사용
이유는 AMEX주식이 1962년 7월부터 포함되기 떄문

Market Factor
CAPM(Sharpe, 1964; Lintner, 1965; Mossin, 1996)에 따르면 기대자산 수익률의 단면변동 = 자산 수익률과 시장 포트폴리오 수익률간의 공분산 함수.

empirical asset pricing에 사용되는 시장 포트폴리오에는 두 가지 주요 proxy 포함
1. CRSP sample의 all U.S market 보통주의 가치 가중 포트폴리오 (French library와 WRDS Fama-French 데이터베이스에서 확인가능)
이 초과수익을 MKT(or MktRf)라 부르며 시장에서 무위험 수익률을 뺸 것을 의미
2. CRSP 가치가중 포트폴리오로 (ADR: American Depository Receipts)를 제외한 CRSP 데이터베이스의 모든 증권 포함
이 포트폴리올르 VWRETD라 표기

MKT와 VWRETD의 차이는 VWRETD에는 미국 기반이 아닌 회사의 주식, 폐쇄형 펀드 및 보통주가 아닌 기타증권이 포함

MKT와 VWRETD는 거의 동일
본 책에서는 MKT 사용
이 포트폴리오의 초과수익률은 market factor의 초과수익률을 대리함

복리 초과 수익률과 누적 로그 초과 수익률을 모두 제시하는 것이 좋음

CAPM

CAPM risk model은 시장 포트폴리오와 관련된 위험에 대한 포트폴리오, 또는 증권의 노출 고려 후 포트폴뢰오, 또는 증권의 평균 비정상 수익률을 계산하는데 사용

알파($\alpha$)라 불리는 비정상 수익률은 시장 포트폴리오의 초과 수익률에 대한 시계열 회귀의 절편계수로 추정
식은 다음과 같음
$$r_{p,t} = \alpha_p + \beta_p MKT_t + \epsilon_{p,t}$$
$r_{p,t}$: excess returns of portfolio being evaluated
$MKT_t$: excess returns of market portfolio

CAPM을 사용해 포트폴리오 수익률을 risk-adjusting 하는 것은 market factor(beta)에 대한 민감도의 단면 차이를 제어하는 것과 유사하다.


## We splite the data sample along two dimension: Stock exchnge and Sector.   

### Decomposing by Stock Exchange

EXCHCD is the exchange code   
1 = NYSE   
2 = AMEX   
3 = NASDAQ   
4 = ARCA   
   
10 = Boston Stock Exchange   
13 = Chicago stock Exxchange   
16 = Pacific Stock Exchange   
17 = Philadelphia Stock Exchange   
19 = Toronto Stock Exchange   
20 = OTC   
   
-2 = delisted   
-1 = suspended   
0 = not NYSE, AMEX, or NASDAQ   

In [3]:
db.describe_table(library='crsp', table='msf')

Approximately 5037353 rows in crsp.msf.


,name,nullable,type,comment
0,cusip,True,VARCHAR(8),None
1,permno,True,INTEGER,None
2,permco,True,INTEGER,None
3,issuno,True,INTEGER,None
4,hexcd,True,SMALLINT,None
5,hsiccd,True,INTEGER,None
6,date,True,DATE,None
7,bidlo,True,"NUMERIC(11, 5)",None
8,askhi,True,"NUMERIC(11, 5)",None
9,prc,True,"NUMERIC(11, 5)",None


Before 1963 July, CRSP doesn't contain AMEX.   
1972 Nov, Nasdaq added at CRSP but NASDAQ stock appears in January 1969.   

### Decomposting by industry sector

These securities can be identified using CRSP’s monthly stock names (msenames) file.   
SHRCD = 10 or 11 is common stock   
SICCD code standard industrial classification code   
SIC CODE 0001-0999: Agriculture, Forestry and Fishing   
SIC CODE 1000-1499: Mining   
SIC CODE 1500-1799: Construction   
SIC CODE 2000-3999: Manufacturing   
SIC CODE 4000-4999: Transportation, Communications, Electric, Gas and Sanitary service   
SIC CODE 5000-5199: Wholesale Trade   
SIC CODE 5200-5999: Retail Trade   
SIC CODE 6000-6799: Finance, Insurance and Real Estate   
SIC CODE 7000-8999: Services   
SIC CODE 9100-9729: Public Administration   

In [4]:
db.describe_table(library='crsp', table='msenames') 

Approximately 113856 rows in crsp.msenames.


,name,nullable,type,comment
0,permno,True,INTEGER,None
1,namedt,True,DATE,None
2,nameendt,True,DATE,None
3,shrcd,True,SMALLINT,None
4,exchcd,True,SMALLINT,None
5,siccd,True,INTEGER,None
6,ncusip,True,VARCHAR(8),None
7,ticker,True,VARCHAR(8),None
8,comnam,True,VARCHAR(35),None
9,shrcls,True,VARCHAR(4),None


## Stock returns and Excess return

In many cases, it is optimal to examine excess stock returns instead of simply the stock return.   
Follow the book, we use risk-free rate from Kenneth French data   
We express excess return as $r_{i,t}$   
   
For the most part, however, the cross-sectional distributions of unadjusted returns look very similar to the distributions of delisting-adjusted returns.   
It is not surprising, therefore, that the choice of whether to use the unadjusted or adjusted returns has very little impact on the results of most empirical analyses.   
That being said, to more accurately reflect the returns an investor would realize by making an investment at the end of month t and holding until the end of month t + 1, we use the delisting-adjusted future excess return (rt+1) as the focal return variable throughout this text.

In [5]:
# Kenneth French Data Library
db.describe_table(library='ff', table='factors_monthly')

Approximately 1174 rows in ff.factors_monthly.


,name,nullable,type,comment
0,date,True,DATE,None
1,mktrf,True,"NUMERIC(8, 6)",None
2,smb,True,"NUMERIC(8, 6)",None
3,hml,True,"NUMERIC(8, 6)",None
4,rf,True,"NUMERIC(7, 5)",None
5,year,True,DOUBLE PRECISION,None
6,month,True,DOUBLE PRECISION,None
7,umd,True,"NUMERIC(8, 6)",None
8,dateff,True,DATE,None


In [6]:
# Kenneth French Data Library
db.describe_table(library='ff', table='factors_daily')

Approximately 25732 rows in ff.factors_daily.


,name,nullable,type,comment
0,date,True,DATE,None
1,mktrf,True,"NUMERIC(8, 6)",None
2,smb,True,"NUMERIC(8, 6)",None
3,hml,True,"NUMERIC(8, 6)",None
4,rf,True,"NUMERIC(7, 5)",None
5,umd,True,"NUMERIC(8, 6)",None


In [7]:
# Get only risk-free rate, market excess return
rf_m = db.raw_sql("""SELECT date, rf, mktrf
                    FROM ff.factors_monthly
                    WHERE date between '1988-01-01' and '2012-12-31' 
                """)

In [8]:
rf_m

,date,rf,mktrf
0,1988-01-01,0.0029,0.0421
1,1988-02-01,0.0046,0.0475
2,1988-03-01,0.0044,-0.0227
3,1988-04-01,0.0046,0.0056
4,1988-05-01,0.0051,-0.0029
...,...,...,...
295,2012-08-01,0.0001,0.0255
296,2012-09-01,0.0001,0.0273
297,2012-10-01,0.0001,-0.0176
298,2012-11-01,0.0001,0.0078


In [9]:
rf_d = db.raw_sql("""SELECT date, rf, mktrf
                    FROM ff.factors_daily
                    WHERE date between '1988-01-01' and '2012-12-31' 
                """)

In [10]:
rf_d

,date,rf,mktrf
0,1988-01-04,0.00015,0.0334
1,1988-01-05,0.00015,0.0122
2,1988-01-06,0.00015,0.0027
3,1988-01-07,0.00015,0.0074
4,1988-01-08,0.00015,-0.0566
...,...,...,...
6297,2012-12-24,0.00001,-0.0024
6298,2012-12-26,0.00001,-0.0054
6299,2012-12-27,0.00001,-0.0011
6300,2012-12-28,0.00001,-0.0100


## Delisted Stock

DLSTDT: date of stock delisting   
DLSTCD: reason for delisting   
DLRET: return of the stock on the delisting date   
Following the Shumway (1997) paper, we consider the stock as delisted if DLSTCD = 500 or 520, between 551 and 573 inclusive, 574, 580, and 584 the stock's return during the delisting month is -30%   
If we can't use the delisting return and DLSTCD is not in the above list, we consider the stock return at delisting date as -100%

## Identifier and characteristic

Get the description of the table crsp.msf   
This table contains monthly stock data for all CRSP-listed securities.   
   
The most used columns are:   
- SHROUT is the number of shares outstanding in thousands
- ALTPRC is the adjusted price. Take absolute value of ALTPRC to get the price 
- MarketCap = SHROUT * ALTPRC / 1000
- Ret is the return of the stock
The exceptions to this are the monthly return when a stock delists from an exchange.   
   
I usally we only consider stocks with EXCHCD = 1, 2, 3, and the others are considered as 0
db.describe_table(library='crsp', table='msf')   

In [15]:
db.describe_table(library='crsp', table='msedelist')

Approximately 37747 rows in crsp.msedelist.


,name,nullable,type,comment
0,permno,True,INTEGER,None
1,dlstdt,True,DATE,None
2,dlstcd,True,SMALLINT,None
3,nwperm,True,INTEGER,None
4,nwcomp,True,INTEGER,None
5,nextdt,True,DATE,None
6,dlamt,True,"NUMERIC(11, 5)",None
7,dlretx,True,"NUMERIC(10, 6)",None
8,dlprc,True,"NUMERIC(11, 5)",None
9,dlpdt,True,DATE,None


In [12]:
db.describe_table(library='crsp', table='msf')

Approximately 5037353 rows in crsp.msf.


,name,nullable,type,comment
0,cusip,True,VARCHAR(8),None
1,permno,True,INTEGER,None
2,permco,True,INTEGER,None
3,issuno,True,INTEGER,None
4,hexcd,True,SMALLINT,None
5,hsiccd,True,INTEGER,None
6,date,True,DATE,None
7,bidlo,True,"NUMERIC(11, 5)",None
8,askhi,True,"NUMERIC(11, 5)",None
9,prc,True,"NUMERIC(11, 5)",None


### More detail

#### Identifier
- Cusip  (VARCHAR(8))       : 8-character identifier for securities, with issuer (6 chars) and issue (2 chars), including dummy codes assigned by CRSP.   
- PERMCO (INTEGER)          : Unique, permanent identifier for companies on CRSP files, with values under 20,000 indicating Nasdaq-assigned numbers.   
- ISSUNO (INTEGER)          : Unique identifier for each Nasdaq-listed security, set to zero if unknown, may change if Nasdaq reassigns the number.   
- HEXCD  (INTEGER)          : Latest exchange code for a security, with 1 for NYSE, 2 for AMEX, and 3 for Nasdaq.   
- HSICCD (INTEGER)          : Latest non-zero Standard Industrial Classification (SIC) Code for a security; zero if not provided by CRSP's data source.   

##### Date
- DATE (DATE)               : Date of the observation, in the form YYYY-MM-DD.

#### Characteristics
- PRC   (NUMERIC(11,5))     : Closing price or negative bid/ask average for a trading day; set to zero if unavailable; monthly data shows last trading date price.   
- VOL   (NUMERIC(10,0))     : Trading volume, expressed in units of one share for daily data and hundreds for monthly data, set to -99 if missing, with zero indicating no trades.   
- RET   (NUMERIC(10,6))     : Holding period return, calculated as (p(t)f(t)+d(t))/p(t')-1, with special codes indicating reasons for missing returns.   
- BID   (NUMERIC(11,5))     : Closing bid price for NYSE, AMEX, and NASDAQ securities; NASDAQ uses inside quotations since July 1980, while NYSE/AMEX uses the last representative quote.   
- ASK   (NUMERIC(11,5))     : Closing ask price for NYSE, AMEX, and NASDAQ securities; NASDAQ uses inside quotations since July 1980, while NYSE/AMEX uses the last representative quote.   
- SHROUT (DOUBLE PRECISION) : Number of publicly held shares, recorded in thousands.   
- BIDLO (NUMERIC(11,5))     : Daily lowest trading or closing bid price, set to zero if unavailable; monthly files show lowest daily price or bid/ask average.   
- ASKHI (NUMERIC(11,5))     : Daily highest trading or closing ask price, set to zero if unavailable; monthly files show highest daily price or bid/ask average.   
- CFACPR(DOUBLE PRECISION)  : Cumulative factor to adjust price.   
- CFACSHR(DOUBLE PRECISION) : Cumulative factor to adjust shares outstanding.   
- ALTPRC(NUMERIC(11,5))     : Alternate monthly price from daily prices, showing the last non-missing price in the month; available only on monthly databases.   
- SPREAD(NUMERIC(10,5))     : Monthly difference between closing bid and ask quotes, set to zero if unavailable, with sign indicating bid/ask status when Closing Price or Bid/Ask Average is zero.   
- ALTPRCDT(DATE)            : Date of the monthly Price Alternate; set to zero if no non-missing prices in the month.   
- RETX(NUMERIC(10,6))       : Holding period return excluding dividends, calculated similarly to RET but typically with dividends (d(t)) set to zero.   

In [16]:
# Use altprc instead of prc to get the adjusted price
sql = """SELECT date, permno, altprc, ret, shrout, vol
        FROM crsp.msf 
        WHERE date between '1988-01-01' and '2012-12-31'
        """
crsp_m = db.raw_sql(sql, date_cols=['date'], chunksize=100000)

In [17]:
crsp_m

,date,permno,altprc,ret,shrout,vol
0,1988-01-29,10001,6.2500,0.063830,992.0,490.0
1,1988-02-29,10001,6.7500,0.080000,992.0,382.0
2,1988-03-31,10001,6.1250,-0.076296,992.0,369.0
3,1988-04-29,10001,-6.3125,0.030612,992.0,121.0
4,1988-05-31,10001,-6.4375,0.019802,992.0,102.0
...,...,...,...,...,...,...
74724,2012-08-31,93436,28.5200,0.040117,105432.0,247781.0
74725,2012-09-28,93436,29.2800,0.026648,105772.0,335868.0
74726,2012-10-31,93436,28.1314,-0.039228,113779.0,178360.0
74727,2012-11-30,93436,33.8200,0.202215,113779.0,224326.0


In [15]:
# There are no altprc in crsp.dsf
crsp_d = db.raw_sql("""SELECT date, permno, prc, ret, shrout, vol
                      FROM crsp.dsf 
                      WHERE date between '1988-01-01' and '2012-12-31'
                  """) 

In [16]:
crsp_d

,date,permno,prc,ret,shrout,vol
0,1988-06-01,10007,NaN,NaN,2064.0,NaN
1,1988-06-02,10007,NaN,NaN,2064.0,NaN
2,1988-01-04,10001,5.875,0.000000,992.0,3200.0
3,1988-01-05,10001,6.375,0.085106,992.0,100.0
4,1988-01-06,10001,6.250,-0.019608,992.0,500.0
...,...,...,...,...,...,...
181098,2012-12-24,93436,34.280,0.008235,113779.0,379504.0
181099,2012-12-26,93436,33.590,-0.020128,113779.0,614673.0
181100,2012-12-27,93436,33.690,0.002977,113779.0,579663.0
181101,2012-12-28,93436,33.220,-0.013951,113779.0,428963.0


### Preprocessing

In [24]:
def convert_types(df):
    """
    Optimizes the data types of a DataFrame.
    
    Args:
        df (pd.DataFrame): The DataFrame to be optimized.
    
    Returns:
        pd.DataFrame: The DataFrame with optimized data types.
    """
    changes = []

    for col in df.columns:
        original_dtype = df[col].dtype
        
        # Convert date columns to datetime type
        if pd.api.types.is_datetime64_any_dtype(df[col]) or 'date' in col.lower():
            df[col] = pd.to_datetime(df[col], errors='coerce')
        
        # Convert integer-like columns to appropriate integer type
        elif pd.api.types.is_integer_dtype(df[col]):
            df[col] = pd.to_numeric(df[col], downcast='integer', errors='coerce')
            # Downcast to smallest possible integer type
            if df[col].dropna().max() <= np.iinfo(np.int16).max and df[col].dropna().min() >= np.iinfo(np.int16).min:
                df[col] = df[col].astype(pd.Int16Dtype())
            elif df[col].dropna().max() <= np.iinfo(np.int32).max and df[col].dropna().min() >= np.iinfo(np.int32).min:
                df[col] = df[col].astype(pd.Int32Dtype())
        
        # Convert float-like columns to appropriate float type
        elif pd.api.types.is_float_dtype(df[col]):
            df[col] = pd.to_numeric(df[col], downcast='float', errors='coerce')
            
            # Check if all values are integers (float ending with .0)
            if all(df[col].dropna() == df[col].dropna().astype(int)):
                df[col] = df[col].astype(pd.Int64Dtype())
                # Downcast to smallest possible integer type
                if df[col].dropna().max() <= np.iinfo(np.int16).max and df[col].dropna().min() >= np.iinfo(np.int16).min:
                    df[col] = df[col].astype(pd.Int16Dtype())
                elif df[col].dropna().max() <= np.iinfo(np.int32).max and df[col].dropna().min() >= np.iinfo(np.int32).min:
                    df[col] = df[col].astype(pd.Int32Dtype())
            else:
                # Downcast to smallest possible float type
                if df[col].dropna().max() <= np.finfo(np.float16).max and df[col].dropna().min() >= np.finfo(np.float16).min:
                    df[col] = df[col].astype(np.float16)
                elif df[col].dropna().max() <= np.finfo(np.float32).max and df[col].dropna().min() >= np.finfo(np.float32).min:
                    df[col] = df[col].astype(np.float32)
        
        # Convert object columns to string type
        elif pd.api.types.is_object_dtype(df[col]):
            df[col] = df[col].astype(str)
        
        # Record changes in column data types
        if df[col].dtype != original_dtype:
            changes.append(f"'{col}' changed from {original_dtype} to {df[col].dtype}")
    
    # Print changes in column data types
    if changes:
        for change in changes:
            print(change)
    else:
        print("No changes were made.")
    
    return df

In [ ]:
crsp_m['altprc'] = ['altprc'].abs() # abs(prc) If there is not a close price, the mid price of bid and ask is used with a negative sign. Change them to a positive value.
crsp_m['mcap'] = crsp_m.altprc * crsp_m.shrout # Market capitalization
crsp_m['size'] = np.log(crsp_m['mcap']) # Size is the log of market capitalization

In [30]:
crsp_m= convert_types(crsp_m)

'mcap' changed from Float32 to float32


In [31]:
crsp_m

,date,permno,altprc,ret,shrout,vol,mcap,size
0,1988-01-29,10001,6.250000,0.063843,992,490,6200.00,8.732305
1,1988-02-29,10001,6.750000,0.080017,992,382,6696.00,8.809266
2,1988-03-31,10001,6.125000,-0.076294,992,369,6076.00,8.712102
3,1988-04-29,10001,6.312500,0.030609,992,121,6262.00,8.742255
4,1988-05-31,10001,6.437500,0.019806,992,102,6386.00,8.761864
...,...,...,...,...,...,...,...,...
74724,2012-08-31,93436,28.520000,0.040131,105432,247781,3006920.75,14.916428
74725,2012-09-28,93436,29.280001,0.026642,105772,335868,3097004.25,14.945946
74726,2012-10-31,93436,28.131399,-0.039215,113779,178360,3200762.50,14.9789
74727,2012-11-30,93436,33.820000,0.202271,113779,224326,3848005.75,15.163066
